In [1]:
from numpy.testing import assert_, assert_equal, assert_almost_equal, assert_allclose, assert_approx_equal, assert_raises, assert_allclose
import numpy as np #import numpy as usual        

##### ME 574 Spring 2021

# Homework 3

1. Fill in python code to implement the function `run_avg(a, rad)` that computes the running average over `(2*r+1)` consecutive entries from the array `a`.

In [2]:
def run_avg(a, r):
    """
    Compute the running average over (2*r+1) consecutive elements of an input array.
    The rad entries at the beginning and end of the output array should agree with those of the input array.
    
    Arguments:
        a: float numpy array of input data
        r: int averaging radius
        
    Returns:
        avg: numpy float array of values over 2*r+1 successive entries in a.
    """
    # YOUR CODE HERE
    sum = 0
    iters = a.size
    start = 0
    end = iters - 1
    ret = np.zeros(iters)
    for i in range(iters):

        start = i-r
        end = i+r

        if(start < 0):
            start = 0
        if(end > iters-1):
            end = iters-1
        tmp = a[start:end+1]
        n = tmp.size 
        ret[i]=tmp.sum()/n
    return ret
    # raise NotImplementedError()

In [3]:
len = 6
nil_array = np.zeros(len)
one_array = np.ones(len)

test_data = np.concatenate((nil_array, one_array))
desired2 = np.array([0. , 0. , 0. , 0. , 0.2, 0.4, 0.6, 0.8, 1. , 1. , 1. , 1. ])
assert_allclose(run_avg(test_data, 2), desired2)
desired3 = np.array([0.        , 0.        , 0.        , 0.14285714, 0.28571429,
       0.42857143, 0.57142857, 0.71428571, 0.85714286, 1.        ,
       1.        , 1.        ])
assert_allclose(run_avg(test_data, 3), desired3)

2. Write python code to implement a parallel version `p_run_avg(a,r)`. Use __global memory__ for this version.

In [4]:
from numba import cuda
from numba import jit, cuda,float32
TPB=32
@cuda.jit
def avg_kernel(avg, d_a, r):
    """
    Kernel function for parallel computation of running average.
    
    Arguments:
        d_out: float device array for storing output
        d_a: float device copy of input data array
        r: int radius
        
    Returns:
        None
    """
    # YOUR CODE HERE
    i = cuda.grid(1)
    n = d_a.shape[0] # 64
    if i < n:
        start = i-r 
        end = i+r
        if start <0:
            start = 0
        if end >= n:
            end = n-1
        j = start
        sum = 0
        while j < end+1:
            sum += d_a[j]
            j+=1
        avg[i] = sum/(end-start+1)

def p_run_avg(a, r):
    """
    Compute the running average over (2*r+1) consecutive elements of an input array in parallel using global memory.
    The rad entries at the beginning and end of the output array should agree with those of the input array.
    
    Arguments:
        a: float numpy array of input data
        r: int averaging radius
        
    Returns:
        avg: numpy float array of values over 2*r+1 successive entries in a.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    n = a.size
    d_a = cuda.to_device(a)
    avg = cuda.device_array(n, dtype = np.float32)
    TPBx = 32
    gridDim = (n+TPBx-1)//TPBx
    blockDim = TPBx
    
    avg_kernel[gridDim, blockDim](avg, d_a, r)
    return avg

In [5]:
TPB = 32
N =  1<<6

nil_array = np.zeros(N)
one_array = np.ones(N)
test_data = np.concatenate((nil_array, one_array))

assert_allclose(p_run_avg(test_data, 2), run_avg(test_data, 2))
assert_allclose(p_run_avg(test_data, 3), run_avg(test_data, 3))

3. Write python code to implement a parallel version `s_run_avg(a,r)`. Use a __shared memory array__ for this version.

In [6]:
import numpy as np #shared.py
from numba import jit, cuda, float32


@cuda.jit
def s_avg_kernel(avg, d_a, r):
    n = d_a.shape[0]    
    N = 516
    RAD = r
    i = cuda.grid(1)
    sh_f = cuda.shared.array(N,dtype=np.float32)
    tIdx = cuda.threadIdx.x #thread index
    shIdx = tIdx + RAD

    sh_f[shIdx] = d_a[i]
    if i >= n:
        sh_f[shIdx] = d_a[n-1]
    cuda.syncthreads() 
    if i<n+RAD:
        sum1 = 0
        j=-RAD
        while j < RAD+1:
            sum1+=sh_f[shIdx+j]
            j+=1
        avg[i] = sum1/(2*r+1)
    


def s_run_avg(a, r):
    """
    Compute the running average over (2*r+1) consecutive elements of an input array in parallel using shared memory.
    The rad entries at the beginning and end of the output array should agree with those of the input array.
    
    Arguments:
        a: float numpy array of input data
        r: int averaging radius
        
    Returns:
        avg: numpy float array of values over 2*r+1 successive entries in a.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    n = a.shape[0]
    d_a = cuda.to_device(a)
    avg = cuda.device_array(n, dtype = np.float32)

    TPBx = 512
    gridDim = (n+TPBx-1)//TPBx
    blockDim = TPBx
   
    s_avg_kernel[gridDim, blockDim](avg, d_a, r)
    return avg

In [7]:
TPB = 32
N =  1<<6

nil_array = np.zeros(N)
one_array = np.ones(N)
test_data = np.concatenate((nil_array, one_array))

assert_allclose(s_run_avg(test_data, 2), run_avg(test_data, 2))
assert_allclose(s_run_avg(test_data, 3), run_avg(test_data, 3))

The remaining problems focus on Laplace's equation, $\nabla^2 u = 0$, which arises frequently is studies of steady-state diffusion (including diffusion of heat as a system evolves toward a sgteady-state temperature distrbution).

In 1D Cartesian coordinates, Laplace's equation simplifies to $\frac{d^2 u}{dx^2} = 0$. A common approach to computing  approximate solutions involves discretizing the domain to a grid of $N+1$ regularly spaced points $x_i = i x + x_o$ and applying a central difference approximation for the derivative to obtain $x_{i-1} -2 x_i + x_{i+1} = 0, i \in [1, \ldots, N]$.

There are 2 distinct ways to proceed toward a solution from this underlying difference equation: treat the collection of $N-1$ equations together as a big linear algebra problem to be solved for the vector of steady-state values on the collection of grid points or use the difference equation as the basis for an iteration scheme to update values at individual grid points. In this problem, we pursue the latter approach and construct an iteration scheme (known as __Jacobi iteration__) as follows:

- Solve the difference equation for $x_i = \frac{1}{2} (x_{i-1} + x_{i+1})$
- Include a superscript to designate the iteration number and update the value of $x_i$ based on the values at the neighboring grid points from the previous iteration: $x_i^{k+1} = \frac{1}{2} (x_{i-1}^k + x_{i+1}^k)$.
- From an initial guess of the solution, update the value at each point (i.e. replace the value with the average of the values at the neighboring grid points) and repeat until the values no longer change significantly during an update step.

 
4. Use the `@cuda.reduce` decorator to write a function `norm_diff(u,v)` that measures the difference between arrays. It will be used to test if the "solution" changes significantly during an update step. Your implementation of `normdiff(u,v)` should implement the $L^1$-norm: $|u-v|_1 = \sum{| u_i - v_i |}$.

In [20]:
# implement the reduction operator
@cuda.reduce
def sum_abs(a,b):
    return abs(a)+abs(b)

# write a reduction `sum_abs(a,b)` operator to sum absolute values
# YOUR CODE HERE
# raise NotImplementedError()
@cuda.jit
def norm_kernal(d_w,d_u,d_v):
    i = cuda.grid(1)
    n = d_u.shape [0]
    if i >= n:
        return
    d_w[i] = abs(d_u[i]-d_v[i])

# implement the L1 norm using the `sum_abs()` reduction
def norm_diff(u,v):
    TPB = 1024
    """
    Compute the L^1 norm of the difference between arrays
    
    Arguments:
        u, v: numpy arrays
        
    Returns:
        value: float norm equal to sum of absolute values of difference between corr. entries
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    n = u.shape[0]
    d_u = cuda.to_device(u)
    d_v = cuda.to_device(v)
    d_w = cuda.device_array(n, dtype = np.float32)
    gridDim = int( np.ceil (n/TPB))
    blockDim = TPB
    norm_kernel[gridDim ,blockDim ](d_w , d_u , d_v)
    value = sum_abs(d_w)
    return value

In [21]:
N = 100
v = np.ones(N)
assert_(sum_abs(v) == np.abs(v).sum())
v = np.array([(-1)**i*(i+1) for i in range(100)])
assert_(sum_abs(v) == np.abs(v).sum())

Below is code for a serial implementation of the function `jacobi_update(u)` that returns $u^{k+1}$ given $u^k$. 
This implementation creates a copy of the input array, writes the updated values into the copy, and returns the updated version of the copy.

In [10]:
def jacobi_update(u):
    """
    Perform one Jacobi iteration step:
    
    Arguments:
        u: numpy array of current values
        
    Returns:
        u_new: float numpy array of updated values (corr. to avg. of neighboring input values)
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    head = u[0]
    end = u[-1]

    n = u.size
    u_new = np.zeros(n)
    for i in range(1,n-1):
            u_new[i] = (u[i-1]+u[i+1])/2
    u_new[-1] = end
    u_new[0] = head
    return u_new

In [11]:
# test to check that jacobi-update works as expected
N = 8
u = np.zeros(N)
u[-1] = 1.
assert_allclose(jacobi_update(u), np.array([0. , 0., 0. , 0. , 0. , 0. , 0.5, 1. ]))

The cell below shows a serial implementation of the function `jacobi_solve(u, tol, max_iters)` that computes an approximate solution of the 1D Laplace equation by performing repeated Jacobi updates until reaching one of the following termination criteria:

- The number of updates reaches a specified maximum value, `max_iters`. (In this case, it prints a messaage warning of failed convergence.)
- The norm of the change in the array caused by an update is less than a specified value, `tol`.

In [12]:
def norm_diff(u_new, u_old):
    n = u_new.shape[0]
    sum = 0
    for i in range(n):
        tmp = abs(u_new[i]-u_old[i])
        sum +=tmp
    # print(sum)
    return sum
def jacobi_solve(u, tol, max_iters):
    """
    Compute an approximate solution of 1D Laplace equation by repeated Jacobi updates.
    
    Arguments:
        u: numpy array of initial guess of solution values
        tol: float L1 norm of change in u at which to stop iterating
        max_iters: maximum number of updates to compute
        
    Returns:
        count: int number of iterations
        sol: numpy float array of approximate solution values
    """
    u_old =  np.copy(u)
    for count in range(1, max_iters+1):
        u_new = jacobi_update(u_old)
        if norm_diff(u_new, u_old)<tol:
            sol = u_new
            return count, sol
        u_old = np.copy(u_new)
        sol = u_new
    return count, sol

In [13]:
N = 11
u = np.zeros(N)
u[-1] = 1.
tol = 0.00005
print(jacobi_solve(u,tol,2))

(2, array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.5 , 1.  ]))


In [14]:
# test jacobi_solve to see if it works as expected
N = 11
u = np.zeros(N)
u[-1] = 1.
tol = 0.00005
assert_allclose(jacobi_solve(u, tol, 1)[1], np.array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0., 0.5 , 1.  ]))
assert_allclose(jacobi_solve(u, tol, 200)[1], 0.1 * np.arange(11), rtol = 0, atol=1e-3)

In [15]:
# test jacobi_solve to see if it works as expected
N = 11
u = np.zeros(N)
u[-1] = 1.
tol = 0.00005
assert_allclose(jacobi_solve(u, tol, 1)[1], np.array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0., 0.5 , 1.  ]))
assert_allclose(jacobi_solve(u, tol, 200)[1], 0.1 * np.arange(11), rtol = 0, atol=1e-3)

5. Code up a parallel implementation of `jacobi_solve()` using __global memory__. To retain the current values (for both later reads and comparison to updated values), use separate arrays for input data and output data. 

Recall that these will need to be device arrays, and there are limits to the memory available on the device. The recommended strategy is to create 2 arrays of the necessary size in global memory and, on subsequent update steps, alternate which array is used for input/output.

Also, remember that copying data back and forth between the host and device would incur a performance penalty, so keep the data on the device side, and only copy the array of final values back to the host side for inspection.

In [16]:
# insert your global memory parallel implementation of jacobi_solve below

TPB = 32

def norm_diff(u, v):
    """
    Compute the L^1 norm of an array
    
    Arguments:
        u: numpy array
        
    Returns:
        value: float norm equal to sum of absolute values of entries
    """
    # YOUR CODE HERE
    # raise NotImplementedError() 
    n = u.shape[0]
    sum = 0
    for i in range(n):
        tmp = abs(u[i]-v[i])
        sum +=tmp
    return sum

@cuda.jit
def update_kernel(d_v, d_u):
    """
    Kernel for parallel jacobi update using global memory.
    """
    # YOUR CODE HERE
    i = cuda.grid(1)
    n = d_u.shape[0]
    head = d_u[0]
    end = d_u[n-1]
    if i >=n:
        d_v[n-1] = d_u[n-1]
        return
    if 1<=i<=n-1:
        d_v[i] = (d_u[i-1]+d_u[i+1])/2
    


   
def p_jacobi_update(u_new,u_old):
    TPB = 512
    """
    Perform one Jacobi iteration step in parallel using global memory.
    
    Arguments:
        d_u: numpy array of current values   0000001
        d_v: numpy array for sotring updated values
        
    Returns:
        None
    """
    # YOUR CODE HERE
    n = u_old.shape[0]
    d_u = cuda.to_device(u_old)
    d_v = cuda.device_array(n, dtype = np.float32)

    TPBx = 512
    gridDim = (n+TPBx-1)//TPBx
    blockDim = TPBx
    update_kernel[gridDim ,blockDim ](d_v, d_u)
    return d_v
  
    # raise NotImplementedError()

def jacobi_solve(u, tol, max_iters):
    """
    Compute an approximate solution of 1D Laplace equation by repeated Jacobi updates.
    This implementation is parallelized using global memory.
    
    Arguments:
        u: numpy array of initial guess of solution values
        tol: float L1 norm of change in u at which to stop iterating
        max_iters: maximum number of updates to compute
        
    Returns:
        count: int number of update iterations
        sol: numpy float array of approximate solution values
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    u_old =  np.copy(u)
    u_new =  np.zeros(u_old.shape[0])

    for count in range(1, max_iters+1):
        u_new = p_jacobi_update(u_new,u_old)
        if norm_diff(u_new, u_old)<tol:
            sol = u_new
            return count, sol
        u_old = np.copy(u_new)
        sol = u_new
    return count, sol

In [17]:
N = 11
u = np.zeros(N)
u[-1] = 1.
tol = 0.00005
count, sol = jacobi_solve(u, tol, 2)
# print(count)
assert_(count==2)
assert_allclose(sol, np.array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.5 , 1.  ]))
count, sol = jacobi_solve(u, tol, 200)
assert_(count == 166)
assert_allclose(sol, 0.1 * np.arange(11), rtol = 0, atol=1e-3)

6. Write code to implement a parallel implementation of `jacobi_solve()` using __shared memory__.

In [18]:
# insert your shared memory parallel implementation of jacobi_solve below

@cuda.jit
def update_kernel(d_v, d_u):
    # RAD = 1
    """
    Kernel for parallel jacobi update using shared memory.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    n = d_u.shape[0]
    i = cuda.grid(1)

    sh_f = cuda.shared.array(512,dtype = np.float32)
    tIdx = cuda.threadIdx.x #thread index
    shIdx = tIdx
    sh_f[shIdx] = d_u[i]
    cuda.syncthreads() 
    
    if  i<n-1:
        d_v[i]=(sh_f[i-1]+sh_f[i+1])/2
        d_v[n-1] = sh_f[n-1]
        d_v[0] = sh_f[0]



  
    
def s_jacobi_update(u_new, u_old):
    """
    Perform one Jacobi iteration step in parallel using shared memory.
    
    Arguments:
        d_u: numpy array of current values
        d_v: numpy array for sotring updated values
        
    Returns:
        None
    """
    # YOUR CODE HERE
    n = u_old.shape[0]
    d_u = cuda.to_device(u_old)
    d_v = cuda.device_array(n, dtype = np.float32)
    TPBx = 512
    gridDim = (n+TPBx-1)//TPBx
    blockDim = TPBx
    update_kernel[gridDim ,blockDim ](d_v, d_u)
    return  d_v.copy_to_host()
    # raise NotImplementedError()

def jacobi_solve(u, tol, max_iters):
    """
    Compute an approximate solution of 1D Laplace equation by repeated Jacobi updates.
    This implementation is parallelized using shared memory.
    
    Arguments:
        u: numpy array of initial guess of solution values
        tol: float L1 norm of change in u at which to stop iterating
        max_iters: maximum number of updates to compute
        
    Returns:
        count: int number of update iterations
        sol: numpy float array of approximate solution values
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    u_old =  np.copy(u)
    u_new =  np.zeros(u_old.shape[0])

    for count in range(1, max_iters+1):
      
        u_new = s_jacobi_update(u_new,u_old)
        if norm_diff(u_new, u_old)<tol:
            sol = u_new
            return count, sol
        u_old = np.copy(u_new)
        sol = u_new
        # print(sol)
    return count, sol

In [19]:
N = 11
u = np.zeros(N)
u[-1] = 1.
tol = 0.00005
count, sol = jacobi_solve(u, tol, 2)
assert_(count == 2)
assert_allclose(sol, np.array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.5 , 1.  ]))
count, sol = jacobi_solve(u, tol, 200)
assert_(count == 166)
assert_allclose(sol, np.linspace(0., 1., 11), rtol = 0, atol=1e-3)